<a href="https://colab.research.google.com/github/alisdghnia/rnn-lstm-interest-rate-prediction/blob/main/RNN_LSTM_InterestRate_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
uploaded = files.upload()

In [3]:
# ir = pd.read_excel(io.BytesIO(uploaded['IR-data.xlsx']))
ir = pd.read_excel('IR-data.xlsx')
ir['delta'] = ir['Interest rate'].diff()
# ir.loc[:, ['Interest rate', 'delta']] = ir.loc[:, ['Interest rate', 'delta']].multiply(0.01)
ir = ir.rename(columns={'Interest rate': 'interest_rate'})
# ir = ir.bfill()
ir = ir.dropna()
ir

,Date,interest_rate,delta
1,2011-01-06,4.13,-0.07
2,2011-01-13,4.08,-0.05
3,2011-01-20,4.05,-0.03
4,2011-01-27,4.09,0.04
5,2011-02-03,4.08,-0.01
...,...,...,...
582,2022-02-24,3.14,-0.01
583,2022-03-03,3.01,-0.13
584,2022-03-10,3.09,0.08
585,2022-03-17,3.39,0.30


In [5]:
train = ir[ir['Date'] < '2021-01-01']
test = ir[ir['Date'] >= '2021-01-01']

x_train = train['interest_rate'].values.reshape(-1, 1)
x_test = test['interest_rate'].values.reshape(-1, 1)

y_train = train['delta'].values.reshape(-1, 1)
y_test = test['delta'].values.reshape(-1, 1)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# RNN Application

In [6]:
model = Sequential()
model.add(SimpleRNN(50, activation='tanh', input_shape=(1, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=10, batch_size=5)
y_pred = model.predict(x_test)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0427
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0041
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0043
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0041
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0048
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0058
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0038
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0035
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0045
Epoch 10/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0053
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


In [7]:
ir4rnn = test.loc[:, ['Date', 'interest_rate']]
ir4rnn.loc[:, 'interest_rate'] = ir4rnn.loc[:, 'interest_rate'] + y_pred.reshape(1, -1)[0]
ir4rnn = ir4rnn.dropna()
ir4rnn

,Date,interest_rate
523,2021-01-07,2.153065
524,2021-01-14,2.223050
525,2021-01-21,2.203053
526,2021-01-28,2.193055
527,2021-02-04,2.203053
...,...,...
582,2022-02-24,3.135923
583,2022-03-03,3.005180
584,2022-03-10,3.085634
585,2022-03-17,3.387307


In [8]:
# test and ir4rnn merge
rnn_merge = pd.merge(test, ir4rnn, on='Date', suffixes=('_original', '_rnn'))
rnn_merge

,Date,interest_rate_original,delta,interest_rate_rnn
0,2021-01-07,2.16,-0.01,2.153065
1,2021-01-14,2.23,0.07,2.223050
2,2021-01-21,2.21,-0.02,2.203053
3,2021-01-28,2.20,-0.01,2.193055
4,2021-02-04,2.21,0.01,2.203053
...,...,...,...,...
59,2022-02-24,3.14,-0.01,3.135923
60,2022-03-03,3.01,-0.13,3.005180
61,2022-03-10,3.09,0.08,3.085634
62,2022-03-17,3.39,0.30,3.387307


In [9]:
rnn_mse = np.mean((rnn_merge['interest_rate_rnn'] - rnn_merge['interest_rate_original'])**2)
rnn_mse

4.382536960462835e-05

# LSTM Application

In [10]:
model = Sequential()
model.add(LSTM(50, activation='tanh', input_shape=(1, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=10, batch_size=5)

y_pred_lstm = model.predict(x_test)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0037
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0036
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0038
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0030
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0043
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0035
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0033
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0035
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0041
Epoch 10/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0045
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  


In [11]:
ir4lstm = test.loc[:, ['Date', 'interest_rate']]
ir4lstm.loc[:, 'interest_rate'] = ir4lstm.loc[:, 'interest_rate'] + y_pred_lstm.reshape(1, -1)[0]
ir4lstm = ir4lstm.dropna()
ir4lstm = ir4lstm.rename(columns={'interest_rate': 'interest_rate_lstm'})
ir4lstm

,Date,interest_rate_lstm
523,2021-01-07,2.140248
524,2021-01-14,2.211810
525,2021-01-21,2.191366
526,2021-01-28,2.181144
527,2021-02-04,2.191366
...,...,...
582,2022-02-24,3.138843
583,2022-03-03,3.006881
584,2022-03-10,3.088111
585,2022-03-17,3.392059


In [12]:
# test and ir4rnn merge
lstm_rnn_merge = pd.merge(rnn_merge, ir4lstm, on='Date')
lstm_rnn_merge

,Date,interest_rate_original,delta,interest_rate_rnn,interest_rate_lstm
0,2021-01-07,2.16,-0.01,2.153065,2.140248
1,2021-01-14,2.23,0.07,2.223050,2.211810
2,2021-01-21,2.21,-0.02,2.203053,2.191366
3,2021-01-28,2.20,-0.01,2.193055,2.181144
4,2021-02-04,2.21,0.01,2.203053,2.191366
...,...,...,...,...,...
59,2022-02-24,3.14,-0.01,3.135923,3.138843
60,2022-03-03,3.01,-0.13,3.005180,3.006881
61,2022-03-10,3.09,0.08,3.085634,3.088111
62,2022-03-17,3.39,0.30,3.387307,3.392059


In [13]:
lstm_mse = np.mean((lstm_rnn_merge['interest_rate_lstm'] - lstm_rnn_merge['interest_rate_original'])**2)
rnn_mse = np.mean((lstm_rnn_merge['interest_rate_rnn'] - lstm_rnn_merge['interest_rate_original'])**2)
lstm_rnn_mse = np.mean((lstm_rnn_merge['interest_rate_lstm'] - lstm_rnn_merge['interest_rate_rnn'])**2)
print(f'lstm_mse: {lstm_mse}')
print(f'rnn_mse: {rnn_mse}')
print(f'lstm_rnn_mse: {lstm_rnn_mse}')

lstm_mse: 0.0002540458686968592
rnn_mse: 4.382536960462835e-05
lstm_rnn_mse: 9.293274444408741e-05
